# Singular Value Decomposition (SVD) - From Scratch

Matrix decomposition, also known as matrix factorization, involves describing a given matrix using its constituent elements. Perhaps the most known and widely used matrix decomposition method is the Singular-Value Decomposition, or SVD. All matrices have an SVD, which makes it more stable than other methods, such as the eigendecomposition. As such, it is often used in a wide array of applications including compressing, denoising, and data reduction.

The Singular-Value Decomposition, or SVD for short, is a matrix decomposition method for reducing a matrix to its constituent parts in order to make certain subsequent matrix calculations simpler.

$A=UDV^{T}$

Where A is the real m x n matrix that we wish to decompose, U is an m x m matrix, D is an m x n diagonal matrix, and V^T is the  transpose of an n x n matrix where T is a superscript. The diagonal values in the D matrix are known as the singular values of the original matrix A. The columns of the U matrix are called the left-singular vectors of A, and the columns of V are called the right-singular vectors of A.

Every rectangular matrix has a singular value decomposition, although the resulting matrices may contain complex numbers and the limitations of floating point arithmetic may cause some matrices to fail to decompose neatly.



## Import libraries

In [1]:
import numpy as np
from scipy.linalg import svd
from numpy.linalg import pinv
from sklearn.decomposition import TruncatedSVD

## Calculate SVD

The SVD is calculated via iterative numerical methods. We will not go into the details of these methods. The SVD can be calculated by calling the svd() function. The function takes a matrix and returns the U, D and V^T elements. The D diagonal matrix is returned as a vector of singular values (d). The V matrix is returned in a transposed form, e.g. V.T.

In [2]:
# define a matrix
A = np.array([[1, 2], [3, 4], [5, 6]])
print(A)

[[1 2]
 [3 4]
 [5 6]]


In [3]:
# SVD
U, d, VT = svd(A)
print(U)
print(d)
print(VT)

[[-0.2298477   0.88346102  0.40824829]
 [-0.52474482  0.24078249 -0.81649658]
 [-0.81964194 -0.40189603  0.40824829]]
[9.52551809 0.51430058]
[[-0.61962948 -0.78489445]
 [-0.78489445  0.61962948]]


In [4]:
# convert singular values d to diagonal matrix
D = np.zeros((A.shape[0], A.shape[1]))
D[:A.shape[1], :A.shape[1]] = np.diag(d)
# D = np.diag(d) if square A matrix
D

array([[9.52551809, 0.        ],
       [0.        , 0.51430058],
       [0.        , 0.        ]])

In [5]:
# reconstruct A
B = U.dot(D.dot(VT))
print(B)

[[1. 2.]
 [3. 4.]
 [5. 6.]]


## SVD for Pseudoinverse

The pseudoinverse is the generalization of the matrix inverse for square matrices to rectangular matrices where the number of rows and columns are not equal. It is also called the the Moore-Penrose Inverse after two independent discoverers of the method or the Generalized Inverse.

$A^{+}=VD^{+}U^{T}$

Where A^+ is the pseudoinverse, D^+ is the pseudoinverse of the diagonal matrix D and U^T is the transpose of U. We can get U and V from the SVD operation. The D^+ can be calculated by creating a diagonal matrix from D, calculating the reciprocal of each non-zero element in D, and taking the transpose if the original matrix was rectangular.

The pseudoinverse provides one way of solving the linear regression equation, specifically when there are more rows than there are columns, which is often the case. NumPy provides the function pinv() for calculating the pseudoinverse of a rectangular matrix.



In [6]:
# define matrix
A = np.array([
	[0.1, 0.2],
	[0.3, 0.4],
	[0.5, 0.6],
	[0.7, 0.8]])
print(A)

[[0.1 0.2]
 [0.3 0.4]
 [0.5 0.6]
 [0.7 0.8]]


In [7]:
# calculate pseudoinverse
B = pinv(A)
print(B)

[[-1.0000000e+01 -5.0000000e+00  8.4040814e-15  5.0000000e+00]
 [ 8.5000000e+00  4.5000000e+00  5.0000000e-01 -3.5000000e+00]]


We can calculate the pseudoinverse manually via the SVD and compare the results to the pinv() function.

First we must calculate the SVD. Next we must calculate the reciprocal of each value in the s array. Then the s array can be transformed into a diagonal matrix with an added row of zeros to make it rectangular. Finally, we can calculate the pseudoinverse from the elements.

In [8]:
# calculate svd
U, d, VT = svd(A)
print(U)
print(d)
print(VT)

[[-0.15248323 -0.82264747 -0.39450102 -0.37995913]
 [-0.34991837 -0.42137529  0.24279655  0.80065588]
 [-0.54735351 -0.0201031   0.69790998 -0.46143436]
 [-0.74478865  0.38116908 -0.5462055   0.04073761]]
[1.42690955 0.06268282]
[[-0.64142303 -0.7671874 ]
 [ 0.7671874  -0.64142303]]


In [9]:
# compute D+
d_inv = 1.0 / d
D_plus = np.zeros(A.shape)
D_plus[:A.shape[1], :A.shape[1]] = np.diag(d_inv)
D_plus

array([[ 0.70081527,  0.        ],
       [ 0.        , 15.95333376],
       [ 0.        ,  0.        ],
       [ 0.        ,  0.        ]])

In [10]:
# calculate pseudoinverse
B = VT.T.dot(D_plus.T).dot(U.T)
print(B)

[[-1.00000000e+01 -5.00000000e+00  1.42578328e-14  5.00000000e+00]
 [ 8.50000000e+00  4.50000000e+00  5.00000000e-01 -3.50000000e+00]]


## SVD for Dimensionality Reduction

Data with a large number of features, such as more features (columns) than observations (rows) may be reduced to a smaller subset of features that are most relevant to the prediction problem. The result is a matrix with a lower rank that is said to approximate the original matrix.

To do this we can perform an SVD operation on the original data and select the top k largest singular values in D. These columns can be selected from D and the rows selected from V^T. An approximate B of the original vector A can then be reconstructed.

$B = U D_k V_k^T $

In natural language processing, this approach can be used on matrices of word occurrences or word frequencies in documents and is called Latent Semantic Analysis or Latent Semantic Indexing.

In practice, we can retain and work with a descriptive subset of the data called T. This is a dense summary of the matrix or a projection.

$T = U D_k$

Further, this transform can be calculated and applied to the original matrix A as well as other similar matrices.

$T = V_k A$

The example below demonstrates data reduction with the SVD.

First a 3×10 matrix is defined, with more columns than rows. The SVD is calculated and only the first two features are selected. The elements are recombined to give an accurate reproduction of the original matrix. Finally the transform is calculated two different ways.




In [11]:
# define a matrix
A = np.array([
	[1,2,3,4,5,6,7,8,9,10],
	[11,12,13,14,15,16,17,18,19,20],
	[21,22,23,24,25,26,27,28,29,30]])
print(A.shape)

(3, 10)


In [12]:
# Singular-value decomposition
U, d, VT = svd(A)
print(U.shape)
print(d.shape)
print(VT.shape)

(3, 3)
(3,)
(10, 10)


In [13]:
# create m x n D matrix
D = np.zeros((A.shape[0], A.shape[1]))
D.shape

(3, 10)

In [14]:
# populate D with n x n diagonal matrix
D[:A.shape[0], :A.shape[0]] = np.diag(d)
D.shape

(3, 10)

In [15]:
# select (k=2) 
n_elements = 2
D = D[:, :n_elements]
VT = VT[:n_elements, :]
print(D.shape)
print(VT.shape)

(3, 2)
(2, 10)


In [16]:
# reconstruct
B = U.dot(D.dot(VT))
print(B)

[[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]
 [11. 12. 13. 14. 15. 16. 17. 18. 19. 20.]
 [21. 22. 23. 24. 25. 26. 27. 28. 29. 30.]]


In [17]:
# transform
T = U.dot(D)
print(T)
print("____________________________________________")
T = A.dot(VT.T)
print(T)

[[-18.52157747   6.47697214]
 [-49.81310011   1.91182038]
 [-81.10462276  -2.65333138]]
____________________________________________
[[-18.52157747   6.47697214]
 [-49.81310011   1.91182038]
 [-81.10462276  -2.65333138]]


The scikit-learn provides a TruncatedSVD class that implements this capability directly.

The TruncatedSVD class can be created in which you must specify the number of desirable features or components to select, e.g. 2. Once created, you can fit the transform (e.g. calculate V^Tk) by calling the fit() function, then apply it to the original matrix by calling the transform() function. The result is the transform of A called T above.

In [18]:
# define array
A = np.array([
	[1,2,3,4,5,6,7,8,9,10],
	[11,12,13,14,15,16,17,18,19,20],
	[21,22,23,24,25,26,27,28,29,30]])
print(A)

[[ 1  2  3  4  5  6  7  8  9 10]
 [11 12 13 14 15 16 17 18 19 20]
 [21 22 23 24 25 26 27 28 29 30]]


In [19]:
# svd
svd = TruncatedSVD(n_components=2)
svd.fit(A)
result = svd.transform(A)
print(result)

[[18.52157747  6.47697214]
 [49.81310011  1.91182038]
 [81.10462276 -2.65333138]]


We can see that the values match those calculated manually above, except for the sign on some values. We can expect there to be some instability when it comes to the sign given the nature of the calculations involved and the differences in the underlying libraries and methods used. This instability of sign should not be a problem in practice as long as the transform is trained for reuse. Analytically the positive or negative solution are two possible solutions, practically they are the same thing.